In [1]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

import random

import numpy as np

import matplotlib.cm as cm

import pickle

import os, sys
    
from denoising_assessment_project.utils import metric_compute, stats_compute, stats_plot

# parameters setup 
from denoising_assessment_project.global_vars import global_vars

%matplotlib notebook

import os

In [2]:
# restoring variables from memory (GT images, indices)
%store -r

In [3]:
# collecting CARE-predicted images into a tensor, ensuring appropriate omage loactions relative to GT images
import glob
import fnmatch
from tifffile import imread

#path at which the predicted images are stored 
path = '/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/predictions/CAREspecial'

predicted_images = sorted(os.listdir(path))
predicted_images_len = len(predicted_images)

print(predicted_images_len)
predicted_images_tensor = torch.Tensor(50,256,256)

for i in range(len(predicted_images)): 
    print(predicted_images[i])
    predicted_images_tensor[i,:,:] = torch.from_numpy(imread(os.path.join(path, predicted_images[i])))
    

50
09_10_00.tif
09_10_01.tif
09_10_02.tif
09_10_03.tif
09_10_04.tif
09_10_05.tif
09_10_06.tif
09_10_07.tif
09_10_08.tif
09_10_09.tif
09_10_10.tif
09_10_11.tif
09_10_12.tif
09_10_13.tif
09_10_14.tif
09_10_15.tif
09_10_16.tif
09_10_17.tif
09_10_18.tif
09_10_19.tif
09_10_20.tif
09_10_21.tif
09_10_22.tif
09_10_23.tif
09_10_24.tif
09_10_25.tif
09_10_26.tif
09_10_27.tif
09_10_28.tif
09_10_29.tif
09_10_30.tif
09_10_31.tif
09_10_32.tif
09_10_33.tif
09_10_34.tif
09_10_35.tif
09_10_36.tif
09_10_37.tif
09_10_38.tif
09_10_39.tif
09_10_40.tif
09_10_41.tif
09_10_42.tif
09_10_43.tif
09_10_44.tif
09_10_45.tif
09_10_46.tif
09_10_47.tif
09_10_48.tif
09_10_49.tif


In [4]:
# opening GT images 
import glob
import fnmatch
from tifffile import imread

#path at which the predicted images are stored 
path = '/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_zoom/noised/'

p = '09'
b = '10'

un_noised_images_list = sorted(glob.glob(os.path.join(path, '{}_{}_*.tif'.format(p,b))))

un_noised_images = torch.Tensor(50,256,256)

for i in range(len(un_noised_images_list)): 
    print(un_noised_images_list[i])
    un_noised_images[i,:,:] = torch.from_numpy(imread(un_noised_images_list[i]))

/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_zoom/noised/09_10_00.tif
/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_zoom/noised/09_10_01.tif
/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_zoom/noised/09_10_02.tif
/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_zoom/noised/09_10_03.tif
/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_zoom/noised/09_10_04.tif
/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_zoom/noised/09_10_05.tif
/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_zoom/noised/09_10_06.tif
/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_zoom/noised/09_10_07.tif
/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_zoom/noised/09_10_08.tif
/Users/miriamczech/Desktop/denoising_assessment/tiffed_data_beam/EVAL_zoom/noised/09_10_09.tif
/Users/miriamczech/Desktop/denoising_assessment/ti

In [5]:
un_noised_images = un_noised_images.unsqueeze(0).unsqueeze(0)
predicted_images_tensor = predicted_images_tensor.unsqueeze(0).unsqueeze(0)

print(un_noised_images.shape)
print(predicted_images_tensor.shape)

torch.Size([1, 1, 50, 256, 256])
torch.Size([1, 1, 50, 256, 256])


In [11]:
def threshold(un_noised_images_tensor): 
    un_noised_images_tensor_max = torch.amax(un_noised_images_tensor, dim=(3,4))
    un_noised_images_tensor_min = torch.amin(un_noised_images_tensor, dim=(3,4))
    un_noised_images_tensor_range = un_noised_images_tensor_max - un_noised_images_tensor_min
    un_noised_images_tensor_threshold = un_noised_images_tensor_range/5 + un_noised_images_tensor_min
    un_noised_images_tensor_threshold_expanded = un_noised_images_tensor_threshold.unsqueeze(3).unsqueeze(4).expand(un_noised_images_tensor.shape)
    
    image_masks = torch.where(un_noised_images_tensor <= un_noised_images_tensor_threshold_expanded, 0, 1)
    return image_masks 

In [12]:
image_masks = threshold(un_noised_images)

In [13]:
# prepare un_noised masked and predicted masked - FOREGROUND

# 0 - background 
# 1 - foreground 

# 1. un_noised masked 
un_noised_foreground = torch.where(image_masks == 0, 0, un_noised_images)
un_noised_foreground[un_noised_foreground == 0] = np.nan

# 1. predicted masked 
predicted_images_tensor_foreground  = torch.where(image_masks == 0, 0, predicted_images_tensor)
predicted_images_tensor_foreground[predicted_images_tensor_foreground== 0] = np.nan

In [14]:
# prepare un_noised masked and predicted masked - BACKGROUND

# 0 - background 
# 1 - foreground 

# 1. un_noised masked 
un_noised_background = torch.where(image_masks == 1, 1, un_noised_images)
un_noised_background[un_noised_background == 1] = np.nan

# 2. predicted masked 
predicted_images_tensor_background  = torch.where(image_masks == 1, 1, predicted_images_tensor)
predicted_images_tensor_background[predicted_images_tensor_background== 1] = np.nan

In [6]:
# CARE: -NRMSE-
# 4a. Computing NRMSE for each GT-CARE_prediction image pair in the data collection
# dims=[m, n, reps]
metric_output = metric_compute.NRMSE_compute(un_noised_images, predicted_images_tensor)

print(metric_output)

mean_NRMSE = torch.mean(metric_output)
std_NRMSE = torch.std(metric_output)

print(mean_NRMSE)
print(std_NRMSE)

tensor([[[0.1724, 0.1711, 0.1714, 0.1713, 0.1691, 0.1716, 0.1723, 0.1724,
          0.1734, 0.1713, 0.1697, 0.1701, 0.1734, 0.1717, 0.1742, 0.1706,
          0.1717, 0.1702, 0.1697, 0.1712, 0.1738, 0.1727, 0.1706, 0.1728,
          0.1711, 0.1708, 0.1726, 0.1707, 0.1731, 0.1731, 0.1705, 0.1716,
          0.1729, 0.1714, 0.1715, 0.1731, 0.1733, 0.1707, 0.1741, 0.1708,
          0.1687, 0.1722, 0.1693, 0.1692, 0.1728, 0.1727, 0.1719, 0.1699,
          0.1731, 0.1723]]])
tensor(0.1716)
tensor(0.0014)


In [15]:
# CARE: -NRMSE FOREGORUND-
# 4a. Computing NRMSE for each GT-CARE_prediction image pair in the data collection
# dims=[m, n, reps]
metric_output = metric_compute.NRMSE_compute(un_noised_foreground, predicted_images_tensor_foreground)

print(metric_output)

mean_NRMSE_f = torch.mean(metric_output)
std_NRMSE_f = torch.std(metric_output)

print(mean_NRMSE_f)
print(std_NRMSE_f)

tensor([[[0.0961, 0.1018, 0.1041, 0.1004, 0.1069, 0.1090, 0.1133, 0.1070,
          0.1090, 0.1064, 0.1043, 0.1090, 0.1135, 0.1023, 0.1054, 0.1113,
          0.1085, 0.1090, 0.1022, 0.1079, 0.1023, 0.1151, 0.1080, 0.1059,
          0.1084, 0.1061, 0.1040, 0.1088, 0.1078, 0.1083, 0.1067, 0.1000,
          0.0997, 0.1092, 0.1100, 0.1034, 0.1106, 0.1013, 0.1038, 0.1135,
          0.1048, 0.1033, 0.1067, 0.1065, 0.1048, 0.1067, 0.0944, 0.1116,
          0.0947, 0.1045]]])
tensor(0.1060)
tensor(0.0045)


In [16]:
# CARE: -NRMSE BACKGROUND-
# 4a. Computing NRMSE for each GT-CARE_prediction image pair in the data collection
# dims=[m, n, reps]
metric_output = metric_compute.NRMSE_compute(un_noised_background, predicted_images_tensor_background)

print(metric_output)

mean_NRMSE_b = torch.mean(metric_output)
std_NRMSE_b = torch.std(metric_output)

print(mean_NRMSE_b)
print(std_NRMSE_b)

tensor([[[0.2083, 0.2235, 0.2386, 0.2309, 0.2438, 0.2593, 0.2603, 0.2508,
          0.2483, 0.2402, 0.2321, 0.2397, 0.2484, 0.2367, 0.2348, 0.2571,
          0.2545, 0.2374, 0.2527, 0.2333, 0.2254, 0.2560, 0.2418, 0.2344,
          0.2393, 0.2427, 0.2300, 0.2465, 0.2551, 0.2544, 0.2538, 0.2300,
          0.2176, 0.2474, 0.2424, 0.2491, 0.2463, 0.2337, 0.2349, 0.2647,
          0.2385, 0.2303, 0.2413, 0.2360, 0.2290, 0.2389, 0.2103, 0.2518,
          0.2020, 0.2322]]])
tensor(0.2397)
tensor(0.0133)


In [18]:
# CARE: -RMSE-
# 4b. Computing RMSE for each image in the data collection
# dims=[m, n, reps]
metric_output = metric_compute.RMSE_compute(un_noised_images, predicted_images_tensor)

print(metric_output)

mean_RMSE = torch.mean(metric_output)
std_RMSE = torch.std(metric_output)

print(mean_RMSE)
print(std_RMSE)

tensor([[[6.2952, 6.2715, 6.1825, 6.2741, 6.2633, 6.2481, 6.1810, 6.2383,
          6.1990, 6.2872, 6.2074, 6.2260, 6.1818, 6.2111, 6.1412, 6.2008,
          6.2696, 6.1738, 6.2637, 6.1776, 6.2576, 6.0589, 6.2605, 6.2746,
          6.2017, 6.2501, 6.2368, 6.2864, 6.1094, 6.2552, 6.2319, 6.3483,
          6.2115, 6.1824, 6.2899, 6.1492, 6.2616, 6.2705, 6.2965, 6.1701,
          6.3206, 6.2406, 6.2416, 6.2066, 6.1657, 6.2927, 6.2774, 6.2126,
          6.2668, 6.2704]]])
tensor(6.2318)
tensor(0.0556)


In [17]:
# CARE: -RMSE FOREGORUND-
# 4a. Computing NRMSE for each GT-CARE_prediction image pair in the data collection
# dims=[m, n, reps]
metric_output = metric_compute.RMSE_compute(un_noised_foreground, predicted_images_tensor_foreground)

print(metric_output)

mean_RMSE_f = torch.mean(metric_output)
std_RMSE_f = torch.std(metric_output)

print(mean_RMSE_f)
print(std_RMSE_f)

tensor([[[10.5349,  9.7761,  9.4703, 10.0547,  9.3467,  9.2148,  8.9534,
           9.4478,  9.3019,  9.5053,  9.4171,  9.2196,  8.9705,  9.6599,
           9.5061,  8.9819,  9.3833,  9.1428,  9.6629,  9.2855,  9.8929,
           8.6536,  9.3403,  9.6949,  9.2600,  9.4180,  9.5423,  9.3849,
           9.1820,  9.3254,  9.3454, 10.3172, 10.2135,  9.1829,  9.3599,
           9.4772,  9.3858,  9.8459,  9.9818,  8.8153,  9.5673,  9.8161,
           9.3454,  9.1681,  9.5679,  9.6596, 10.6187,  8.9785, 10.7436,
           9.7108]]])
tensor(9.5126)
tensor(0.4392)


In [16]:
# CARE: -RMSE BACKGROUND-
# 4a. Computing NRMSE for each GT-CARE_prediction image pair in the data collection
# dims=[m, n, reps]
metric_output = metric_compute.RMSE_compute(un_noised_background, predicted_images_tensor_background)

print(metric_output)

mean_RMSE_b = torch.mean(metric_output)
std_RMSE_b = torch.std(metric_output)

print(mean_RMSE_b)
print(std_RMSE_b)

tensor([[[0.2083, 0.2235, 0.2386, 0.2309, 0.2438, 0.2593, 0.2603, 0.2508,
          0.2483, 0.2402, 0.2321, 0.2397, 0.2484, 0.2367, 0.2348, 0.2571,
          0.2545, 0.2374, 0.2527, 0.2333, 0.2254, 0.2560, 0.2418, 0.2344,
          0.2393, 0.2427, 0.2300, 0.2465, 0.2551, 0.2544, 0.2538, 0.2300,
          0.2176, 0.2474, 0.2424, 0.2491, 0.2463, 0.2337, 0.2349, 0.2647,
          0.2385, 0.2303, 0.2413, 0.2360, 0.2290, 0.2389, 0.2103, 0.2518,
          0.2020, 0.2322]]])
tensor(0.2397)
tensor(0.0133)


In [8]:
# CARE: -PSNR- 
# 4c. Computing PSNR for each image in the data collection
# dims=[m, n, reps]
metric_output = metric_compute.PSNR_compute(un_noised_images, predicted_images_tensor)

print(metric_output)

mean_PSNR = torch.mean(metric_output)
std_PSNR = torch.std(metric_output)

print(mean_PSNR)
print(std_PSNR)

tensor([[[35.4301, 33.4695, 32.3987, 32.7679, 31.6971, 31.2084, 30.3307,
          32.2920, 30.9974, 32.4872, 32.8859, 31.5609, 31.6727, 33.7174,
          32.9647, 30.6872, 31.9704, 31.8701, 32.4635, 31.9722, 33.7278,
          30.0273, 31.6823, 33.0267, 31.7823, 32.6724, 33.5609, 31.3075,
          31.5101, 31.9131, 31.5769, 33.2067, 34.1936, 31.4730, 31.6758,
          32.2596, 31.6099, 33.3688, 33.2082, 30.5698, 32.0216, 33.0701,
          31.8887, 31.8959, 32.8729, 32.4840, 35.4989, 31.1340, 35.8390,
          33.0540]]])
tensor(32.3791)
tensor(1.2320)


In [20]:
# CARE: -PSNR FOREGORUND-
# 4a. Computing NRMSE for each GT-CARE_prediction image pair in the data collection
# dims=[m, n, reps]
metric_output = metric_compute.PSNR_compute(un_noised_foreground, predicted_images_tensor_foreground)

print(metric_output)

mean_PSNR_f = torch.mean(metric_output)
std_PSNR_f = torch.std(metric_output)

print(mean_PSNR_f)
print(std_PSNR_f)

tensor([[[30.9577, 29.6137, 28.6948, 28.6715, 28.2200, 27.8336, 27.1122,
          28.6868, 27.4724, 28.8970, 29.2659, 28.1509, 28.4387, 29.8813,
          29.1698, 27.4687, 28.4681, 28.4596, 28.6979, 28.4324, 29.7495,
          26.9314, 28.2071, 29.2475, 28.3002, 29.1110, 29.8671, 27.8269,
          27.9713, 28.4447, 28.0574, 28.9885, 29.8741, 28.0365, 28.2233,
          28.5025, 28.0941, 29.4498, 29.2061, 27.4710, 28.4210, 29.1358,
          28.3827, 28.5075, 29.0562, 28.7616, 30.9330, 27.9353, 31.1569,
          29.2548]]])
tensor(28.7140)
tensor(0.9094)


In [25]:
# CARE: -PSNR BACKGROUND-
# 4a. Computing NRMSE for each GT-CARE_prediction image pair in the data collection
# dims=[m, n, reps]
metric_output = metric_compute.PSNR_compute(un_noised_background, predicted_images_tensor_background)

print(metric_output)

mean_PSNR_b = torch.mean(metric_output)
std_PSNR_b = torch.std(metric_output)

print(mean_PSNR_b)
print(std_PSNR_b)

RuntimeError: The size of tensor a (57190) must match the size of tensor b (57266) at non-singleton dimension 0

In [9]:
# CARE: -SSIM- 
# 4c. Computing SSIM for each image in the data collection
# dims=[m, n, reps]
metric_output = metric_compute.SSIM_compute(un_noised_images, predicted_images_tensor)

print(metric_output)

mean_SSIM = torch.mean(metric_output)
std_SSIM = torch.std(metric_output)

print(mean_SSIM)
print(std_SSIM)

tensor([[[0.9980, 0.9954, 0.9947, 0.9861, 0.9935, 0.9957, 0.9931, 0.9984,
          0.9824, 0.9976, 0.9971, 0.9911, 0.9994, 0.9994, 0.9990, 0.9951,
          0.9994, 0.9948, 0.9975, 0.9905, 0.9971, 0.9871, 0.9934, 0.9988,
          0.9911, 0.9993, 0.9993, 0.9921, 0.9957, 0.9990, 0.9973, 0.9915,
          0.9983, 0.9941, 0.9975, 0.9943, 0.9968, 0.9985, 0.9972, 0.9988,
          0.9918, 0.9939, 0.9941, 0.9925, 0.9923, 0.9974, 0.9977, 0.9967,
          0.9987, 0.9971]]])
tensor(0.9953)
tensor(0.0037)


In [23]:
# CARE: -SSIM FOREGORUND-
# 4a. Computing NRMSE for each GT-CARE_prediction image pair in the data collection
# dims=[m, n, reps]
metric_output = metric_compute.SSIM_compute(un_noised_foreground, predicted_images_tensor_foreground)

print(metric_output)

mean_SSIM_f = torch.mean(metric_output)
std_SSIM_f = torch.std(metric_output)

print(mean_SSIM_f)
print(std_SSIM_f)

tensor([[[0.9954, 0.9922, 0.9917, 0.9806, 0.9907, 0.9940, 0.9907, 0.9972,
          0.9781, 0.9961, 0.9951, 0.9879, 0.9994, 0.9993, 0.9980, 0.9929,
          0.9994, 0.9925, 0.9957, 0.9868, 0.9947, 0.9840, 0.9904, 0.9977,
          0.9876, 0.9991, 0.9990, 0.9890, 0.9933, 0.9983, 0.9954, 0.9867,
          0.9960, 0.9918, 0.9955, 0.9913, 0.9948, 0.9967, 0.9949, 0.9982,
          0.9883, 0.9904, 0.9915, 0.9895, 0.9887, 0.9952, 0.9948, 0.9952,
          0.9961, 0.9951]]])
tensor(0.9931)
tensor(0.0047)


In [24]:
# CARE: -SSIM BACKGROUND-
# 4a. Computing NRMSE for each GT-CARE_prediction image pair in the data collection
# dims=[m, n, reps]
metric_output = metric_compute.SSIM_compute(un_noised_background, predicted_images_tensor_background)

print(metric_output)

mean_SSIM_b = torch.mean(metric_output)
std_SSIM_b = torch.std(metric_output)

print(mean_SSIM_b)
print(std_SSIM_b)

tensor([[[0.9016, 0.8423, 0.8687, 0.8760, 0.8456, 0.8290, 0.8313, 0.8383,
          0.8505, 0.8452, 0.8732, 0.8098, 0.8518, 0.8574, 0.8501, 0.8358,
          0.8299, 0.8622, 0.8173, 0.8772, 0.8845, 0.7925, 0.8281, 0.8595,
          0.8116, 0.8160, 0.8755, 0.8468, 0.8786, 0.8355, 0.8657, 0.8852,
          0.8916, 0.8615, 0.8305, 0.8512, 0.8243, 0.8854, 0.8823, 0.8186,
          0.8373, 0.8635, 0.8584, 0.8315, 0.8693, 0.8393, 0.8911, 0.8471,
          0.9084, 0.8521]]])
tensor(0.8523)
tensor(0.0258)
